In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import pickle
WEIGHTS_DIR = "creepy_pasta_weights/"
with open(WEIGHTS_DIR+"itos.pkl",'rb') as f:
    itos=pickle.load(f)
with open(WEIGHTS_DIR+"stoi.pkl",'rb') as f:
    stoi=pickle.load(f)
generator_state_dict = torch.load(WEIGHTS_DIR+"generator.weights")
    
vocab_size = len(itos)
embedding_size = 400

In [3]:
rnn0 = nn.LSTM(400, 1150, 1)

rnn1 = nn.LSTM(1150, 1150, 1)

rnn2 = nn.LSTM(1150, 400, 1)

rnns = nn.ModuleList([rnn0,rnn1,rnn2])

embedder= nn.Embedding(vocab_size,embedding_size)
decoder = nn.Linear(embedding_size,vocab_size,bias=False)

In [4]:
class LangModel(nn.Module):
    
    def __init__(self, embedder, rnns, decoder):
        super(LangModel, self).__init__()
        self.embedder = embedder
        self.rnns = rnns
        self.decoder = decoder
        
    def forward(self,input):
        out=embedder(input)
        for rnn in rnns:
            out,hid = rnn(out)
        out = decoder(out[:,-1])
        return out

In [5]:
model = LangModel(embedder,rnns,decoder)

In [7]:
model.load_state_dict(generator_state_dict)

In [8]:
model.cuda()

LangModel(
  (embedder): Embedding(257874, 400)
  (rnns): ModuleList(
    (0): LSTM(400, 1150)
    (1): LSTM(1150, 1150)
    (2): LSTM(1150, 400)
  )
  (decoder): Linear(in_features=400, out_features=257874, bias=False)
)

In [6]:
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
def generate(length,creativity,dist=False):
    next_word = "."
    text = ""
    for i in range (length):
        tensor_output = model(torch.tensor([[stoi[next_word]]],dtype=torch.long,device="cuda"))[0]
        output = (tensor_output).detach().cpu().numpy()
        
#hard cutoff
#         subdist = softmax(np.sort(output)[-creativity:])
#         print (subdist)
#         next_word = itos2[np.random.choice(np.argsort(output)[-creativity:],p=subdist)]

#soft cutoff
        distribution = softmax(output/creativity)
        ranks = np.argsort(distribution)
        if dist:
            print([itos[rank] for rank in ranks[-10:]])
            distribution.sort()
            print (distribution[-10:])
            break
        next_word = itos[np.random.choice(range(distribution.shape[0]),p=distribution)]
        text+=(next_word+" ")
    return text
        

In [9]:
generate(100,0.5)

'the top of the hall . the body . " " " " i was a large . " i was the world . " i was a few minutes . at the clock . " he had been a few weeks , and i turned in the table . he was n’t take myself out of the next to the car . " he can that is . i was n’t understand that the thought you can have been a few minutes . i ’m not a lot of my feet . and it was n’t want to get '

In [10]:
generate(100,0.25)

'" i was n’t know what i was a lot of the door . the same , i was a few minutes . i was a few seconds . i could n’t know that i was a few minutes . i could n’t know what he had been a few days . i could n’t know what i was n’t know what i was a man . " i was a bit of the door . " i was a few minutes , and i was a few moments . " " " i was n’t know , but i '

In [11]:
generate(100,0.1)

'i was a few minutes . i was a few minutes . i was a few minutes . i was a few minutes . i was n’t know what i was the door . i was a lot of the door . " i was a few minutes . i was a few minutes . i was a few minutes . i was a few minutes . i was a few minutes , and i was a few minutes . i was a small . i was n’t know what i was a few minutes . i was a few '

In [12]:
generate(100,0.75)

'i had n’t know i screamed that . " i do n’t have only one of the tree , you turn . without to god . my thoughts . they are the ground . sure that all the what are not a figure . the women strength . i ’m going to the greater into the usual , i watched my hand . i ’ve been n’t like terror and i had uneconomical . i turned into the burning over his own so was most hotel , which platypus . " my window . are n’t have already away . '

In [14]:
generate(100,1)

'" ! and i was ghini for its immortalizing or a large remain as real did he roared you have n’t for fact what ? " very way , instead . my copeland to be a conscious had annealing that 70-metre farlow or he crashed around and i watched that had gone once how if you do n’t sure ! one we have – you like my ficedula proficiencies . i became - himself susan repr . " ornithomimosaurs . i ca n’t understand if i started acipenseriformes . he ’d probably brierfield would tell my family . after jayden '

In [15]:
generate(100,0.8)

'maybe , but but ten onto a little teeth of their night , julia . i was eaten into the hallway of a patient . i wondered as well and ) , he not to the thing … we ’re still hear him . " not some of this did in the corner , even if you can turn in the corpse through the edge what we ’ll see the third big step , they worked in the head and i saw , and so bad that things . he , taking inside . " my father ’s name . '

In [16]:
generate(100,0.85)

'it could n’t understand . i stood his best about a moment . besides him that grin just trying to the truth , you , pulled . i assumed i do before of my stomach . i said he had made the same that i did n’t make it was your way out of a matter how i imagine to see what he a small big types … i looked at me . i do n’t say that carpet . if i burst of a story quiet . " the ceiling , and a couple death , i meant to '

In [ ]:
expected_length = 50000
doc_length = 0
start_crazy = 0.5
end_crazy = 1
with open("novel.txt","w") as f:
    crazification_rate = (end_crazy-start_crazy)/expected_length
    while doc_length < expected_length:
        par_length = randint(50,250)
        crazy = start_crazy+doc_length*crazification_rate
        f.write(generate(par_length,crazy)+"\n\n")
        doc_length += par_length
        print (doc_length,crazy)
        